In [ ]:
from vitessce import (
    VitessceWidget,
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    SnapWrapper,
)
from os.path import join
from scipy.io import mmread
import pandas as pd
import numpy as np

In [ ]:
def get_hg_view_config(port):
    return {
        "uid": 'A',
        "chromInfoPath": '//s3.amazonaws.com/pkerp/data/hg38/chromSizes.tsv',
        "tracks": {
          "top": [
            {
              "type": "horizontal-chromosome-labels",
              "server": "https://higlass.io/api/v1",
              "tilesetUid": "NyITQvZsS_mOFNlz5C2LJg",
              "uid": "chromosome-labels",
              "options": {
                "color": '#808080',
                "stroke": 'black',
                "fontSize": 12,
                "fontIsLeftAligned": False,
                "showMousePosition": True,
                "mousePositionColor": '#ff00ff',
              },
              "height": 30
            },
            {
              "type": "horizontal-gene-annotations",
              "server": "https://higlass.io/api/v1",
              "tilesetUid": "P0PLbQMwTYGy-5uPIQid7A",
              "uid": "gene-labels",
              "options": {
                "name": 'Gene Annotations (hg38)',
                "fontSize": 10,
                "labelPosition": 'hidden',
                "labelLeftMargin": 0,
                "labelRightMargin": 0,
                "labelTopMargin": 0,
                "labelBottomMargin": 0,
                "minHeight": 24,
                "geneAnnotationHeight": 16,
                "geneLabelPosition": 'outside',
                "geneStrandSpacing": 4,
                "showMousePosition": True,
                "mousePositionColor": '#ff00ff',
                "plusStrandColor": 'silver',
                "minusStrandColor": 'silver',
                "labelColor": 'black',
                "trackBorderWidth": 0,
                "trackBorderColor": 'black',
              },
              "height": 70
            },
          ] + [
            {
              "type": "horizontal-bar",
              "uid": f"bar-track-{i}",
              "data": {
                "type": "zarr-multivec",
                "url": f"http://localhost:{port}/A/0/genomic/profiles.zarr",
                "row": i,
              },
              "options": {
                "name": f"Cluster {i}",
                "labelColor": "black",
                "labelPosition": "topLeft",
                "axisPositionHorizontal": "right",
                "barFillColor": "silver",
                "valueScaling": "linear",
                "trackBorderWidth": 0,
                "trackBorderColor": "black",
                "labelTextOpacity": 0.4,
                "barOpacity": 1,
                "align": "bottom",
                "labelLeftMargin": 0,
                "labelRightMargin": 0,
                "labelTopMargin": 0,
                "labelBottomMargin": 0,
                "labelShowResolution": False,
                "axisLabelFormatting": "scientific",
                "labelShowAssembly": False
              },
              "width": 568,
              "height": 42
            } for i in range(17)
          ],
          "left": [],
          "center": [],
          "right": [],
          "bottom": [],
        },
        "layout": {
          "w": 12,
          "h": 12,
          "x": 0,
          "y": 0,
          "moved": False,
          "static": False,
        },
    }

In [ ]:
mtx = mmread(join('data', 'snapatac', 'filtered_cell_by_bin.mtx'))
barcodes_df = pd.read_csv(join('data', 'snapatac', 'barcodes.txt'), header=None)
bins_df = pd.read_csv(join('data', 'snapatac', 'bins.txt'), header=None)
clusters_df = pd.read_csv(join('data', 'snapatac', 'umap_coords_clusters.csv'), index_col=0)

w = SnapWrapper(mtx, barcodes_df, bins_df, clusters_df)

In [ ]:
port = 9085

In [ ]:
vc = VitessceConfig(name='HiGlass', description='HiGlass')
dataset = vc.add_dataset(name='HBM485.TBWH.322').add_object(w)
higlass = vc.add_view(dataset, cm.HIGLASS)
scatter = vc.add_view(dataset, cm.SCATTERPLOT, mapping = "UMAP")
cell_sets = vc.add_view(dataset, cm.CELL_SETS)

higlass.set_props(hgViewConfig=get_hg_view_config(port))

vc.layout(higlass | (scatter / cell_sets));

In [ ]:
vw = VitessceWidget(vc, port=port, height=900)
vw